<a href="https://colab.research.google.com/github/ayoubajermoune/Windows-VPS/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [3]:
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import csv
import time


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
traders_list = []
def dianmic_webset():
    url_list = ['https://my.litefinance.org/traders?type=profit', 'https://my.litefinance.org/traders?type=risk']
    for url in url_list:
        browser.get(url)
        for _ in range(1):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
        content = BeautifulSoup(browser.page_source , 'html.parser')
        traders = content.find_all('div' , {'class' : 'card_trader'})
        for trader in traders:
          profitability = trader.find('div' , {'class' : 'user_info'}).find('div' , {'class' : 'data'}).find('div' , {'class' : 'data_value'})
          profitability = profitability.text.strip().replace('%', '')
          if float(profitability) >= 10:
            user_info = trader.find('div' , {'class' : 'user_info'}).find('div' , {'class' : 'title'})
            id = trader.find('a' , {'class' : 'link'}).get('href')
            result = {
                'trader name': user_info.text.strip(),
                'id': id.strip().replace('/traders/info?id=', ''),
                'Profitability (All time) %' : profitability
            }
            traders_list.append(result)
    with open('traders.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['trader name', 'id' , 'Profitability (All time) %']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for trader in traders_list:
            writer.writerow(trader)
dianmic_webset()


In [24]:

def annual_rates(id:int,path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        detail_chart_range = content.find('div' , {'class' : 'trader_detail_chart'})
        bars = content.find_all('div' , {'class' : 'bars'})[0].find_all('div' , {'class' : 'bar'})
        Date = []
        for bar in bars:
          date = bar.find('div' , {'class' : 'data_label'}).text.strip().replace(' ','').replace('\n' , ' ').split()
          date = ' '.join(date[1:3])
          result = {
              'Date':date
          }
          Date.append(result)

        Profitability = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip().replace('%' , '')
          result = {
              'Profitability %' : value
          }
          Profitability.append(result)

        bars = content.find_all('div' , {'class' : 'bars'})[1].find_all('div' , {'class' : 'bar'})
        Risk = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip().replace('risk' , '')
          result = {
              'Risk' : value
          }
          Risk.append(result)

        bars = content.find_all('div' , {'class' : 'bars'})[2].find_all('div' , {'class' : 'bar'})
        Copying = []
        for bar in bars:
          value = bar.find('div' , {'class' : 'data_value'}).text.strip()
          result = {
              'Copying' : value
          }
          Copying.append(result)

        df = pd.DataFrame({
            'Date': Date,
            'Risk': Risk,
            'Copying': Copying,
            'Profitability %': Profitability
        })

        # Format Date column
        df['Date'] = df['Date'].apply(lambda x: x['Date'])
        df['Risk'] = df['Risk'].apply(lambda x: x['Risk'])
        df['Copying'] = df['Copying'].apply(lambda x: x['Copying'])
        df['Profitability %'] = df['Profitability %'].apply(lambda x: x['Profitability %'])


        # Set Date as index
        df.set_index('Date', inplace=True)

        # Write DataFrame to CSV
        csv_file = f"{path}/annual rates.csv"
        df.to_csv(csv_file)


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def popular_tools(id:int , path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        trader_detail_tools = content.find_all('div', {'class': 'panel_inner'})[2]
        content_row = trader_detail_tools.find_all('div', {'class': 'content_row_inner'})
        detail_tools = []
        for row in content_row:
            title = row.find('div', {'class': 'title'})
            trades = row.find_all('div', {'class': 'data'})[1].find('div', 'data_value')
            profitable = row.find_all('div', {'class': 'data'})[2].find('div', 'data_value')
            result = {
                'title': title.text.strip().replace('_o', ''),
                'trades': trades.text.strip(),
                'Profitable %': profitable.text.strip().replace('%', '')
            }
            detail_tools.append(result)

        field_names = ['title', 'trades', 'Profitable %']
        with open(f'{path}/popular tools.csv', mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=field_names)
            writer.writeheader()
            for detail in detail_tools:
                writer.writerow(detail)

In [ ]:
def Portfolio_composition(id:int , path:str):
    url_trader = f'https://my.litefinance.org/traders/info?id={id}'
    r = requests.get(url_trader)
    if r.status_code == 200:
        content = BeautifulSoup(r.text, 'html.parser')
        trader_detail = content.find('div' , {'class' : 'trader_detail_tools_chart_data'}).find_all('div' , {'class' : 'data'})
        Detail_name = []
        Detail_value = []
        for detail in trader_detail:
          mareket = detail.find('div' , {'class' : 'data_label'})
          precentage = detail.find('div' , {'class' : 'data_value'})
          Detail_name.append(f'{mareket.text.strip()} %')
          Detail_value.append(precentage.text.strip().replace('%', ''))

        with open(f'{path}/Portfolio composition.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(Detail_name)
            writer.writerow(Detail_value)

In [25]:
import pandas as pd
import os

traders = pd.read_csv('traders.csv')

for index, row in traders.iterrows():
    trader_name = row['trader name']
    trader_id = row['id']
    new_folder_path = f'/content/traders info/{trader_name}'
    try:
      os.makedirs(new_folder_path)
    except:
      pass

    annual_rates(id =trader_id , path = f'{new_folder_path}')

    popular_tools(id =trader_id , path = f'{new_folder_path}')

    Portfolio_composition(id =trader_id , path = f'{new_folder_path}')







